In [1]:
from libquantum import gaussian_elimination, Symlfsr, eval_bit
from tqdm.auto import tqdm

In [2]:
LEAK = False
# LEAK = True

with open("quantum.jpg", "rb") as f:
    crib_pln = f.read()
with open("quantum.jpg.enc", "rb") as f:
    crib_enc = f.read()
crib_n = len(crib_pln)
print(f"{crib_n=}")

with open("flag.enc", "rb") as f:
    ciphertext = f.read()
ciphertext_n = len(ciphertext)
print(f"{ciphertext_n=}")

if LEAK:
    with open("seq_leak.txt", "r") as f:
        seq_leak_bin = f.read().strip()
    seq_n = len(seq_leak_bin)
    print(f"{seq_n=}")

    seq_leak = []
    for i in range(0, seq_n, 8):
        seq_leak.append(int(seq_leak_bin[i:i+8], 2))
    seq_leak = bytes(seq_leak)

crib_n=10764
ciphertext_n=114819


In [3]:
def bxor(a, b):
    return bytes(i ^ j for i, j in zip(a, b))

crib_otp = bxor(crib_pln, crib_enc)
crib_otp[:20].hex()

if LEAK:
    crib_otp_leak = bxor(
        bxor(
            seq_leak[0:crib_n],
            seq_leak[crib_n:crib_n*2],
        ),
        seq_leak[crib_n*2:crib_n*3]
    )
    assert crib_otp == crib_otp_leak

In [4]:
symlfsr = Symlfsr.init_sym()
relations = {
    i: 0
    for i in range(Symlfsr.n)
}
solutions = {
    i: (crib_otp[i // 8] >> (7-(i % 8))) & 1
    for i in range(Symlfsr.n)
}

for rep in range(3):
    for pos in tqdm(range(crib_n * 8)):
        lfsr_out = symlfsr.next()
        if pos < Symlfsr.n:
            relations[pos] ^= lfsr_out

  0%|          | 0/86112 [00:00<?, ?it/s]

  0%|          | 0/86112 [00:00<?, ?it/s]

  0%|          | 0/86112 [00:00<?, ?it/s]

In [5]:
initial_state = gaussian_elimination(relations, solutions)

if LEAK:
    for k, v in initial_state.items():
        assert int(seq_leak_bin[k]) == v

  0%|          | 0/19937 [00:00<?, ?it/s]

  0%|          | 0/19937 [00:00<?, ?it/s]

In [6]:
symlfsr = Symlfsr.init_with_state(initial_state)

for rep in range(3):
    for pos in tqdm(range(crib_n * 8)):
        lfsr_out = symlfsr.next()

ciphertext_otp_bin = [0 for _ in range(ciphertext_n * 8)]

for rep in range(3):
    for pos in tqdm(range(ciphertext_n * 8)):
        lfsr_out = symlfsr.next()
        ciphertext_otp_bin[pos] ^= lfsr_out

  0%|          | 0/86112 [00:00<?, ?it/s]

  0%|          | 0/86112 [00:00<?, ?it/s]

  0%|          | 0/86112 [00:00<?, ?it/s]

  0%|          | 0/918552 [00:00<?, ?it/s]

  0%|          | 0/918552 [00:00<?, ?it/s]

  0%|          | 0/918552 [00:00<?, ?it/s]

In [7]:
ciphertext_otp = []
for i in range(0, len(ciphertext_otp_bin), 8):
    ciphertext_otp.append(int(
        "".join(str(j) for j in ciphertext_otp_bin[i:i+8])
    , 2))
ciphertext_otp = bytes(ciphertext_otp)

if LEAK:
    ciphertext_otp_leak = bxor(
        bxor(
            seq_leak[crib_n*3:crib_n*3+ciphertext_n],
            seq_leak[crib_n*3+ciphertext_n:crib_n*2+ciphertext_n*2],
        ),
        seq_leak[crib_n*3+ciphertext_n*2:crib_n*3+ciphertext_n*3]
    )

plaintext = bxor(ciphertext, ciphertext_otp)
with open("dec.bin", "wb") as f:
    f.write(plaintext)